In [3]:
import langchain
print (langchain.__version__)

1.2.6


In [ ]:
import os
from google import genai
from langchain_google_genai import ChatGoogleGenerativeAI

# 1. Setup your key
os.environ["GEMINI_API_KEY"] = "XXX"
# 2. Initialize the model 
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

## Load Documents from CSV file (Each row is one document)

In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="outdoorclothing.csv")
docs = loader.load()

print (docs[0].metadata)
print ("---------------------------------------")
print (docs[0].page_content)


{'source': 'outdoorclothing.csv', 'row': 0}
---------------------------------------
: 0
name: Women's Campside Oxfords
description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. 

Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. 

Specs: Approx. weight: 1 lb.1 oz. per pair. 

Construction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. 

Questions? Please contact us for any inquiries.


## Chunking (Optional)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,      # Roughly how many characters
    chunk_overlap = 50,    # Repeated text between chunks to keep context
)

chunks = text_splitter.split_documents(docs)

print (f"Original Docs: {len(docs)}")
print (f"Chunks: {len(chunks)}")
print (chunks[0].page_content)
print ("---------------------------------------")
print (chunks[1].page_content)


Original Docs: 1000
Chunks: 6313
: 0
name: Women's Campside Oxfords
---------------------------------------
description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on.


##### Note: If each CSV row already fits comfortably in the LLM context (that is one idea), then chunking usually doesn't help and can make the result worst. Chunking only starts to make sense when a single row contains long free-text that's too big to retrieve/use as one piece.

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Embeddings model
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Create vector store + index (store) the documents
vector_store = InMemoryVectorStore(embedding = embedding_model)
vector_store.add_documents(docs)

# Chunking + Create vector store + index (store) the documents (OPTIONAL)
# splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, overlap_chunk=200, add_start_index=True)
# splits = splitter.split_documents(docs)
# vector_store = InMemoryVectorStore(embedding = embedding_model)
# vector_store.add_documents(documents=splits)

['c0cdd1a4-2f47-43d1-8aee-70158774a8c3',
 '28d9cb03-2a8f-4fed-b048-5f2c70baf3df',
 '21e0e835-a58a-4d80-a264-60d6fa89e2bc',
 '64d30cf2-f9ed-44f2-8960-9b623f705353',
 'bfba9dd9-e35b-4027-9134-09ff26d90eac',
 'f7a25765-5ec9-41ff-8b69-4713e710e35f',
 '8ffebb5d-f0f3-4878-b0e1-803d43023018',
 '6dcef3ac-9a73-4aca-b564-a041d5469d37',
 '8cbded1d-6631-4fb8-a8a3-24d1c0397484',
 'f8062456-2d1c-483c-85d1-d2f2ff6a3adc',
 'c468c503-605d-48a0-a850-b07405957d12',
 'd597c89c-535e-4396-9b05-8466949e3625',
 '344c9cea-9a90-4a8f-89ba-2f6dd9394c76',
 '25705ab6-226f-4313-b99c-7c0fd0844db3',
 'aabd1787-d914-4138-a6f8-57482960ac67',
 '1e1be8eb-2616-4235-b4c8-7518f90aae3c',
 'c9fe4440-d947-49a5-a1e3-0bb6f280d985',
 '77ec6c64-2706-4fc5-8250-63da1823dcaf',
 '85278bec-0f1f-4958-ac70-be19e3e98ee7',
 'a46ffa2b-f677-4665-8453-5796e25c9e1d',
 'c3d56cc5-e9ae-423c-8397-5d4b1bf914a5',
 '805b6748-b7a0-4727-b63f-1fa047d539c7',
 'ff49f8b2-1962-4ebd-a52f-0f32874d690f',
 '69b1db61-c459-4b50-8893-1c78c80a638c',
 '55ee1fd6-eaab-

In [37]:
# Quick sanity check: Similarity Search

print ("-------------Reference Document--------------------")
print (docs[100].page_content)
print ("---------------------------------------")


results = vector_store.similarity_search("What is the brand of the shoe that features a drop-in heel and is mesh?", k=3)
print(results[0].page_content)

-------------Reference Document--------------------
: 100
name: Men's OluKai Nohea Breeze Slip-Ons
description: These men's mesh slip-on shoes are designed with a versatile drop-in heel and ultrabreathable mesh upper, giving them a sleek, distinguished look with handsewn seam stitching and embroidered details. Elastic gore side panels provide a comfortable fit, while moisture-wicking Dri-Lex microfiber lining keeps feet cool and dry. An all-weather nonmarking rubber outsole with heavy-gauge sidewall stitch provides support and traction. Imported. Questions? Please contact customer service.
---------------------------------------
: 100
name: Men's OluKai Nohea Breeze Slip-Ons
description: These men's mesh slip-on shoes are designed with a versatile drop-in heel and ultrabreathable mesh upper, giving them a sleek, distinguished look with handsewn seam stitching and embroidered details. Elastic gore side panels provide a comfortable fit, while moisture-wicking Dri-Lex microfiber lining ke

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.chains import RetrievalQA 

# Create Retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":4}
    )

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)


print ("\n-------------Reference Document--------------------")
print (docs[100].page_content[:200])
print ("\n-------------Answer--------------------")
result = qa.invoke({"query":"Which shoe features a drop-in heel and is mesh?"})
print (result["result"])


print ("\n-------------Reference Document--------------------")
print (docs[200].page_content[:200])
print ("\n-------------Answer--------------------")
result = qa.invoke({"query":"Which women's pants is suited for rigorous hikes?"})
print (result["result"])



-------------Reference Document--------------------
: 100
name: Men's OluKai Nohea Breeze Slip-Ons
description: These men's mesh slip-on shoes are designed with a versatile drop-in heel and ultrabreathable mesh upper, giving them a sleek, distinguished

-------------Answer--------------------
The Kids' OluKai Sea Breeze Mesh Slip-Ons and the Men's OluKai Nohea Breeze Slip-Ons both feature a drop-in heel and are mesh.

-------------Reference Document--------------------
: 200
name: Adventure Trekking Pants
description: Our most technical women’s hiking pants are expertly constructed for rigorous hikes. Durable, high-performance ripstop fabric, articulated knees and t

-------------Answer--------------------
Adventure Trekking Pants are expertly constructed for rigorous hikes.
